## Transformer代码理解

### 引入

In [60]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

import os
# 指定使用GPU进行训练
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


### 引入数据

In [61]:
# 手动输入了两对德语→英语的句子
# 为减低难度，没有引用数据集

# S: Symbol that shows starting of decoding input
# E: Symbol that shows starting of decoding output
# P: Symbol that will fill in blank sequence if current batch data size is short than time steps
# 句子的输入部分
sentences = [
        # enc_input           dec_input         dec_output
        ['ich mochte ein bier P', 'S i want a beer .', 'i want a beer . E'],
        ['ich mochte ein cola P', 'S i want a coke .', 'i want a coke . E']
]

# Padding Should be Zero
# 构建词表
src_vocab = {'P' : 0, 'ich' : 1, 'mochte' : 2, 'ein' : 3, 'bier' : 4, 'cola' : 5}
src_vocab_size = len(src_vocab)

tgt_vocab = {'P' : 0, 'i' : 1, 'want' : 2, 'a' : 3, 'beer' : 4, 'coke' : 5, 'S' : 6, 'E' : 7, '.' : 8}
idx2word = {i: w for i, w in enumerate(tgt_vocab)}
tgt_vocab_size = len(tgt_vocab)

src_len = 5 # enc_input max sequence length
tgt_len = 6 # dec_input(=dec_output) max sequence length


### 数据预处理

In [62]:
# 对数据进行手动编辑 每个字的索引
def make_data(sentences):
    enc_inputs, dec_inputs, dec_outputs = [], [], []
    for i in range(len(sentences)):
      enc_input = [[src_vocab[n] for n in sentences[i][0].split()]] # [[1, 2, 3, 4, 0], [1, 2, 3, 5, 0]]
      dec_input = [[tgt_vocab[n] for n in sentences[i][1].split()]] # [[6, 1, 2, 3, 4, 8], [6, 1, 2, 3, 5, 8]]
      dec_output = [[tgt_vocab[n] for n in sentences[i][2].split()]] # [[1, 2, 3, 4, 8, 7], [1, 2, 3, 5, 8, 7]]

      enc_inputs.extend(enc_input)
      dec_inputs.extend(dec_input)
      dec_outputs.extend(dec_output)

    return torch.LongTensor(enc_inputs), torch.LongTensor(dec_inputs), torch.LongTensor(dec_outputs)

enc_inputs, dec_inputs, dec_outputs = make_data(sentences)


# 构建数据集
class MyDataSet(Data.Dataset):
    def __init__(self, enc_inputs, dec_inputs, dec_outputs):
        super(MyDataSet, self).__init__()
        self.enc_inputs = enc_inputs
        self.dec_inputs = dec_inputs
        self.dec_outputs = dec_outputs
  
    def __len__(self):
        return self.enc_inputs.shape[0]
  
    def __getitem__(self, idx):
        return self.enc_inputs[idx], self.dec_inputs[idx], self.dec_outputs[idx]

loader = Data.DataLoader(MyDataSet(enc_inputs, dec_inputs, dec_outputs), 2, True)

### 位置编码 Positional Encoding

In [63]:
## 位置编码的实现其实很简单，直接对照着公式去敲代码就可以，下面这个代码只是其中一种实现方式，还有通过for循环进行实现
## 从理解来讲，需要注意的就是偶数和奇数在公式上有一个共同部分，我们使用log函数把次方拿下来，方便计算；
## pos代表的是单词在句子中的索引，这点需要注意；比如max_len是128个，那么索引就是从0，1，2，...,127
## 假设demodel是512，2i那个符号中i从0取到了255，那么2i对应取值就是0,2,4...510

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # 创建常数 'pe' 矩阵，其值取决于 pos and i
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)## 这里需要注意的是pe[:, 0::2]这个用法，就是从0开始到最后面，补长为2，其实代表的就是偶数位置
        pe[:, 1::2] = torch.cos(position * div_term)## 这里需要注意的是pe[:, 1::2]这个用法，就是从1开始到最后面，补长为2，其实代表的就是奇数位置
        ## 上面代码获取之后得到的pe:[max_len*d_model]
        
        ## 下面这个代码之后，我们得到的pe形状是：[max_len*1*d_model]
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        '''
        x: [seq_len, batch_size, d_model]
        '''
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
    

### MASK

#### Pad Mask
针对句子不够长，加了 pad

Mask在变TRM中起着重要作用。
它有两个目的：
+ 在编码器和解码器中：只要输入句子中有填充，就可以将注意力输出归零。
+ 在解码器中：防止解码器在预测下一个单词时已经知道全部句子。

In [64]:
## 比如说，现在的句子长度是5，在后面注意力机制的部分，我们在计算出来QK转置除以根号之后，softmax之前，我们得到的形状为len_input * len *input  
## 所以这里需要有一个同等大小形状的矩阵，告诉我哪个位置是PAD部分，之后在计算softmax之前会把这里置为无穷大；

def get_attn_pad_mask(seq_q, seq_k):
    '''
    seq_q: [batch_size, seq_len]
    seq_k: [batch_size, seq_len]
    seq_len could be src_len or it could be tgt_len
    seq_len in seq_q and seq_len in seq_k maybe not equal
    '''
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # [batch_size, 1, len_k], False is masked
    # 这句的作用是返回一个大小和 seq_k 一样的 tensor，只不过里面的值只有 True 和 False。   
    # 如果 seq_k 某个位置的值等于 0，那么对应位置就是 True，否则即为 False。
    # 例如：输入为 seq_data = [1, 2, 3, 4, 0]，seq_data.data.eq(0) 就会返回 [False, False, False, False, True] 
    
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # [batch_size, len_q, len_k]
    # 得到的矩阵形状是batch_size * len_q * len_k，我们是对k中的pad符号进行标识
    # seq_q 和 seq_k 不一定一致，在交互注意力，q来自解码端，k来自编码端，所以告诉模型编码这边pad符号信息就可以，解码端的pad信息在交互注意力层是没有用到的


#### Subsequence Mask
Decoder部分：用来屏蔽未来时刻单词的信息

In [65]:
# 首先通过 np.ones() 生成一个全 1 的方阵
# 然后通过 np.triu() 生成一个上三角矩阵  下三角全为0
def get_attn_subsequence_mask(seq):
    '''
    seq: [batch_size, tgt_len]
    '''
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    subsequence_mask = np.triu(np.ones(attn_shape), k=1) # Upper triangular matrix
    
    subsequence_mask = torch.from_numpy(subsequence_mask).byte()
    return subsequence_mask # [batch_size, tgt_len, tgt_len]

# Mask与Scaled Scores进行相加以后，之后通过softmax,上三角变为0，即得到这个矩阵为每个字之间的权重。
# 具体见下一步

#### ScaledDotProductAttention

In [66]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        '''
        输入进来的维度：
        Q: [batch_size, n_heads, len_q, d_k]
        K: [batch_size, n_heads, len_k, d_k]
        V: [batch_size, n_heads, len_v(=len_k), d_v]
        attn_mask: [batch_size, n_heads, seq_len, seq_len]
        '''
        
        # 经过matmul函数得到的scores形状是 : [batch_size, n_heads, len_q, len_k]
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) 
        # 第一步将 Q 和 K 转置相乘 得到的 scores
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is True.
        # scores 和 attn_mask 相加，把一些需要屏蔽的信息屏蔽掉
        # attn_mask 和 scores 的维度相同
        
        attn = nn.Softmax(dim=-1)(scores)
        # mask 之后，对 scores 进行 softmax ，softmax之后被mask的地方基本就是0，对q的单词不起作用
        context = torch.matmul(attn, V) # [batch_size, n_heads, len_q, d_v]
        # 与 V 相乘，得到每个单词的 context vector
        return context, attn


### 多头注意力MultiHeadAttention

In [67]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        ## 输入进来的QKV是相等的，我们会使用映射linear做一个映射得到参数矩阵Wq, Wk,Wv
        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=False)
        self.fc = nn.Linear(n_heads * d_v, d_model, bias=False)
    def forward(self, input_Q, input_K, input_V, attn_mask):
        # 首先映射分头，然后计算atten_scores，然后计算atten_value;
        '''
        input_Q: [batch_size, len_q, d_model]
        input_K: [batch_size, len_k, d_model]
        input_V: [batch_size, len_v(=len_k), d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        '''
        
        residual, batch_size = input_Q, input_Q.size(0)
        # (B, S, D) -proj-> (B, S, D_new) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        # 下面这个就是先映射，后分头；一定要注意的是q和k分头之后维度是一致的，所以一看这里都是d_k
        Q = self.W_Q(input_Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # Q: [batch_size, n_heads, len_q, d_k]
        K = self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # K: [batch_size, n_heads, len_k, d_k]
        V = self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # V: [batch_size, n_heads, len_v(=len_k), d_v]

        ## 输入进行的attn_mask形状是 [batch_size x len_q x len_k]
        ## 然后经过下面这个代码得到 新的attn_mask : [batch_size x n_heads x len_q x len_k]，就是把pad信息重复了n个头上
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size, n_heads, seq_len, seq_len]

        # 然后计算 ScaledDotProductAttention 这个函数得到 context 和 attn
        '''
        context: [batch_size, n_heads, len_q, d_v]
        attn: [batch_size, n_heads, len_q, len_k]
        '''
        context, attn = ScaledDotProductAttention()(Q, K, V, attn_mask)
        context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v) # context: [batch_size, len_q, n_heads * d_v]
        output = self.fc(context) # [batch_size, len_q, d_model]
        return nn.LayerNorm(d_model).cuda()(output + residual), attn
    
    
## 完整代码中一定会有三处地方调用 MultiHeadAttention()
## Encoder Layer 调用一次，传入的 input_Q、input_K、input_V 全部都是 enc_inputs；
## Decoder Layer 中两次调用，第一次传入的全是 dec_inputs，第二次传入的分别是 dec_outputs，enc_outputs

### FeedForward Layer

In [68]:
# 两层线性映射，并用激活函数激活
# 前馈层只是加深了我们的网络，使用线性层来分析注意力层输出中的模式。

class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=False)
        )
    def forward(self, inputs):
        '''
        inputs: [batch_size, seq_len, d_model]
        '''
        residual = inputs
        output = self.fc(inputs)
        # FeedForward 残差连接与 Layer Normalization
        return nn.LayerNorm(d_model).cuda()(output + residual) # [batch_size, seq_len, d_model]
# 归一化在深度神经网络中非常重要。
# 它可以防止层中值的范围变化太大，这意味着模型训练更快并且具有更好的泛化能力。

### Encoder Layer

In [69]:
# 包含两个部分，多头注意力机制和前馈神经网络，即将以上组件拼起来，就是一个完整的 Encoder Layer
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        # 构建一个具有一个多头注意力层和一个前馈层的编码器层
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        '''
        enc_inputs: [batch_size, src_len, d_model]
        enc_self_attn_mask: [batch_size, src_len, src_len]
        '''
        
        # 下面这个就是做自注意力层，输入是enc_inputs，最初始的QKV矩阵是等同于这个输入的。
        # enc_outputs: [batch_size, src_len, d_model], attn: [batch_size, n_heads, src_len, src_len]
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size, src_len, d_model]
        return enc_outputs, attn

### Encoder

In [70]:
# 包含三个部分：词向量embedding，位置编码部分，注意力层及后续的前馈神经网络
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(src_vocab_size, d_model)## 这个是去定义生成一个矩阵，大小是 src_vocab_size * d_model
        self.pos_emb = PositionalEncoding(d_model)## 位置编码情况，这里是固定的正余弦函数，也可以使用类似词向量的nn.Embedding获得一个可以更新学习的位置编码
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])# 这里有多个层，使用ModuleList对多个encoder进行堆叠
        # 用 for 循环以嵌套的方式，将上一次 Encoder Layer 的输出作为下一次 Encoder Layer 的输入

    def forward(self, enc_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        '''
        ## 下面这个代码通过src_emb，进行索引定位，
        enc_outputs = self.src_emb(enc_inputs) # [batch_size, src_len, d_model]
        ## 位置编码，把两者相加放入到了这个函数里面，
        enc_outputs = self.pos_emb(enc_outputs.transpose(0, 1)).transpose(0, 1) # [batch_size, src_len, d_model]
        ## get_attn_pad_mask是为了得到句子中pad的位置信息，给到模型后面，在计算自注意力和交互注意力的时候去掉pad符号的影响，
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs) # [batch_size, src_len, src_len]
        enc_self_attns = []
        for layer in self.layers:
            # enc_outputs: [batch_size, src_len, d_model], enc_self_attn: [batch_size, n_heads, src_len, src_len]
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attns.append(enc_self_attn)
        return enc_outputs, enc_self_attns

### Decoder Layer

In [71]:
class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        # 调用两次 MultiHeadAttention
        self.dec_self_attn = MultiHeadAttention()
        self.dec_enc_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, dec_inputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):
        '''
        dec_inputs: [batch_size, tgt_len, d_model]
        enc_outputs: [batch_size, src_len, d_model]
        dec_self_attn_mask: [batch_size, tgt_len, tgt_len]
        dec_enc_attn_mask: [batch_size, tgt_len, src_len]
        '''

        ## 第一次调用 MultiHeadAttention是计算 Decoder Input 的 self-attention，得到输出 dec_outputs；
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)# dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len]

        ## 将 dec_outputs 作为生成 Q 的元素，enc_outputs 作为生成 K 和 V 的元素，再调用一次 MultiHeadAttention，
        ## 得到的是 Encoder 和 Decoder Layer 之间的 context vector。
        dec_outputs, dec_enc_attn = self.dec_enc_attn(dec_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)# dec_outputs: [batch_size, tgt_len, d_model], dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
        ## 将 dec_outptus 做一次维度变换，然后返回
        dec_outputs = self.pos_ffn(dec_outputs) # [batch_size, tgt_len, d_model]
        return dec_outputs, dec_self_attn, dec_enc_attn

### Decoder

In [72]:
# 与 encoder 部分基本一样
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.tgt_emb = nn.Embedding(tgt_vocab_size, d_model)
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])

    def forward(self, dec_inputs, enc_inputs, enc_outputs):
        '''
        dec_inputs: [batch_size, tgt_len]
        enc_intpus: [batch_size, src_len]
        enc_outputs: [batsh_size, src_len, d_model]
        '''
        dec_outputs = self.tgt_emb(dec_inputs) # [batch_size, tgt_len, d_model]
        dec_outputs = self.pos_emb(dec_outputs.transpose(0, 1)).transpose(0, 1).cuda() # [batch_size, tgt_len, d_model]
        
        # Decoder 中不仅要把 "pad"mask 掉，还要 mask 未来时刻的信息
        ## get_attn_pad_mask 自注意力层的时候的pad 部分
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs).cuda() # [batch_size, tgt_len, tgt_len]
        ## get_attn_subsequent_mask 做的是自注意层的mask部分，就是当前单词之后看不到，使用一个上三角为1的矩阵
        dec_self_attn_subsequence_mask = get_attn_subsequence_mask(dec_inputs).cuda() # [batch_size, tgt_len, tgt_len]
        ## 两矩阵相加；torch.gt(a, value) 的意思是，将 a 中各个位置上的元素和 value 比较，若大于 value，则该位置取 1，否则取 0，这里和0作比较
        dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequence_mask), 0).cuda() # [batch_size, tgt_len, tgt_len]

        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs) # [batc_size, tgt_len, src_len]

        dec_self_attns, dec_enc_attns = [], []
        for layer in self.layers:
            # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
            dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            dec_self_attns.append(dec_self_attn)
            dec_enc_attns.append(dec_enc_attn)
        return dec_outputs, dec_self_attns, dec_enc_attns

### Transformer

注意相关的调用：
从 Encoder 到 Embedding、 PositionalEncoding、EncoderLayer，EncoderLayer 到 MultiHeadAttention、PoswiseFeedForwardNet 等

In [73]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.encoder = Encoder().cuda()## 编码层
        self.decoder = Decoder().cuda()## 解码层
        self.projection = nn.Linear(d_model, tgt_vocab_size, bias=False).cuda()## 输出层 d_model 是我们解码层每个token输出的维度大小，之后会做一个 tgt_vocab_size 大小的softmax
    def forward(self, enc_inputs, dec_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        dec_inputs: [batch_size, tgt_len]
        '''
        # tensor to store decoder outputs
        # outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)
        
        # enc_outputs: [batch_size, src_len, d_model], enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len]
        # enc_inputs作为输入，enc_outputs是主要输出
        enc_outputs, enc_self_attns = self.encoder(enc_inputs)
        # dec_outpus: [batch_size, tgt_len, d_model], dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [n_layers, batch_size, tgt_len, src_len]
        # dec_outputs 是decoder主要输出，用于后续的linear映射； 
        # dec_self_attns类比于enc_self_attns 是查看每个单词对decoder中输入的其余单词的相关性；dec_enc_attns是decoder中每个单词对encoder中每个单词的相关性；
        dec_outputs, dec_self_attns, dec_enc_attns = self.decoder(dec_inputs, enc_inputs, enc_outputs)
        # dec_outputs做映射到词表大小
        dec_logits = self.projection(dec_outputs) # dec_logits: [batch_size, tgt_len, tgt_vocab_size]
        # 可以理解为，一个句子，这个句子有 batch_size*tgt_len 个单词，每个单词有 tgt_vocab_size 种情况，取概率最大者
        return dec_logits.view(-1, dec_logits.size(-1)), enc_self_attns, dec_self_attns, dec_enc_attns

### 模型 & 损失函数 & 优化器

In [74]:
# Transformer Parameters 相关参数
d_model = 512  # Embedding Size
d_ff = 2048 # FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_layers = 6  # number of Encoder of Decoder Layer
n_heads = 8  # number of heads in Multi-Head Attention

model = Transformer().cuda()
# 交叉熵损失；这里设置了一个参数 ignore_index=0，因为 "pad" 这个单词的索引为 0，设置以后就不会计算 "pad" 的损失
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.99)


### 训练

In [75]:
for epoch in range(1000):
    for enc_inputs, dec_inputs, dec_outputs in loader:
      '''
      enc_inputs: [batch_size, src_len]
      dec_inputs: [batch_size, tgt_len]
      dec_outputs: [batch_size, tgt_len]
      '''
      enc_inputs, dec_inputs, dec_outputs = enc_inputs.cuda(), dec_inputs.cuda(), dec_outputs.cuda()
      # outputs: [batch_size * tgt_len, tgt_vocab_size]
      outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)
      loss = criterion(outputs, dec_outputs.view(-1))
      print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

Epoch: 0001 loss = 2.395966
Epoch: 0002 loss = 2.118629
Epoch: 0003 loss = 1.982401
Epoch: 0004 loss = 1.694296
Epoch: 0005 loss = 1.597510
Epoch: 0006 loss = 1.355717
Epoch: 0007 loss = 1.207036
Epoch: 0008 loss = 0.938905
Epoch: 0009 loss = 0.739689
Epoch: 0010 loss = 0.538913
Epoch: 0011 loss = 0.427697
Epoch: 0012 loss = 0.265680
Epoch: 0013 loss = 0.225038
Epoch: 0014 loss = 0.172337
Epoch: 0015 loss = 0.186000
Epoch: 0016 loss = 0.143255
Epoch: 0017 loss = 0.126766
Epoch: 0018 loss = 0.114316
Epoch: 0019 loss = 0.090952
Epoch: 0020 loss = 0.071264
Epoch: 0021 loss = 0.058963
Epoch: 0022 loss = 0.048296
Epoch: 0023 loss = 0.032709
Epoch: 0024 loss = 0.029582
Epoch: 0025 loss = 0.024144
Epoch: 0026 loss = 0.027236
Epoch: 0027 loss = 0.021866
Epoch: 0028 loss = 0.024376
Epoch: 0029 loss = 0.017331
Epoch: 0030 loss = 0.017472
Epoch: 0031 loss = 0.012050
Epoch: 0032 loss = 0.007073
Epoch: 0033 loss = 0.007765
Epoch: 0034 loss = 0.010372
Epoch: 0035 loss = 0.006508
Epoch: 0036 loss = 0

Epoch: 0297 loss = 0.000032
Epoch: 0298 loss = 0.000034
Epoch: 0299 loss = 0.000038
Epoch: 0300 loss = 0.000015
Epoch: 0301 loss = 0.000056
Epoch: 0302 loss = 0.000032
Epoch: 0303 loss = 0.000055
Epoch: 0304 loss = 0.000034
Epoch: 0305 loss = 0.000015
Epoch: 0306 loss = 0.000007
Epoch: 0307 loss = 0.000024
Epoch: 0308 loss = 0.000014
Epoch: 0309 loss = 0.000035
Epoch: 0310 loss = 0.000029
Epoch: 0311 loss = 0.000021
Epoch: 0312 loss = 0.000036
Epoch: 0313 loss = 0.000051
Epoch: 0314 loss = 0.000048
Epoch: 0315 loss = 0.000045
Epoch: 0316 loss = 0.000032
Epoch: 0317 loss = 0.000026
Epoch: 0318 loss = 0.000011
Epoch: 0319 loss = 0.000032
Epoch: 0320 loss = 0.000014
Epoch: 0321 loss = 0.000028
Epoch: 0322 loss = 0.000018
Epoch: 0323 loss = 0.000019
Epoch: 0324 loss = 0.000019
Epoch: 0325 loss = 0.000013
Epoch: 0326 loss = 0.000012
Epoch: 0327 loss = 0.000033
Epoch: 0328 loss = 0.000013
Epoch: 0329 loss = 0.000029
Epoch: 0330 loss = 0.000023
Epoch: 0331 loss = 0.000010
Epoch: 0332 loss = 0

Epoch: 0590 loss = 0.000003
Epoch: 0591 loss = 0.000004
Epoch: 0592 loss = 0.000003
Epoch: 0593 loss = 0.000002
Epoch: 0594 loss = 0.000003
Epoch: 0595 loss = 0.000002
Epoch: 0596 loss = 0.000003
Epoch: 0597 loss = 0.000002
Epoch: 0598 loss = 0.000003
Epoch: 0599 loss = 0.000002
Epoch: 0600 loss = 0.000006
Epoch: 0601 loss = 0.000002
Epoch: 0602 loss = 0.000003
Epoch: 0603 loss = 0.000004
Epoch: 0604 loss = 0.000003
Epoch: 0605 loss = 0.000003
Epoch: 0606 loss = 0.000004
Epoch: 0607 loss = 0.000005
Epoch: 0608 loss = 0.000002
Epoch: 0609 loss = 0.000003
Epoch: 0610 loss = 0.000003
Epoch: 0611 loss = 0.000002
Epoch: 0612 loss = 0.000003
Epoch: 0613 loss = 0.000009
Epoch: 0614 loss = 0.000003
Epoch: 0615 loss = 0.000003
Epoch: 0616 loss = 0.000006
Epoch: 0617 loss = 0.000006
Epoch: 0618 loss = 0.000003
Epoch: 0619 loss = 0.000002
Epoch: 0620 loss = 0.000004
Epoch: 0621 loss = 0.000003
Epoch: 0622 loss = 0.000002
Epoch: 0623 loss = 0.000003
Epoch: 0624 loss = 0.000003
Epoch: 0625 loss = 0

Epoch: 0885 loss = 0.000002
Epoch: 0886 loss = 0.000003
Epoch: 0887 loss = 0.000002
Epoch: 0888 loss = 0.000003
Epoch: 0889 loss = 0.000002
Epoch: 0890 loss = 0.000004
Epoch: 0891 loss = 0.000001
Epoch: 0892 loss = 0.000002
Epoch: 0893 loss = 0.000002
Epoch: 0894 loss = 0.000002
Epoch: 0895 loss = 0.000002
Epoch: 0896 loss = 0.000002
Epoch: 0897 loss = 0.000003
Epoch: 0898 loss = 0.000003
Epoch: 0899 loss = 0.000002
Epoch: 0900 loss = 0.000002
Epoch: 0901 loss = 0.000003
Epoch: 0902 loss = 0.000002
Epoch: 0903 loss = 0.000002
Epoch: 0904 loss = 0.000001
Epoch: 0905 loss = 0.000002
Epoch: 0906 loss = 0.000003
Epoch: 0907 loss = 0.000002
Epoch: 0908 loss = 0.000002
Epoch: 0909 loss = 0.000002
Epoch: 0910 loss = 0.000002
Epoch: 0911 loss = 0.000002
Epoch: 0912 loss = 0.000002
Epoch: 0913 loss = 0.000002
Epoch: 0914 loss = 0.000001
Epoch: 0915 loss = 0.000002
Epoch: 0916 loss = 0.000002
Epoch: 0917 loss = 0.000002
Epoch: 0918 loss = 0.000003
Epoch: 0919 loss = 0.000002
Epoch: 0920 loss = 0

### 测试

正常的测试流程：
翻译器通过运行一个循环来工作。
我们首先对英文句子进行编码。
然后我们向解码器提供 ‘S’ 标记索引和编码器输出。
解码器对第一个单词进行预测，然后我们将其与 ‘S’ 标记一起添加到解码器输入中。
我们重新运行循环，获取下一个预测并将其添加到解码器输入，直到我们到达 ‘E’ 标记，让我们知道它已经完成翻译。

In [76]:
def greedy_decoder(model, enc_input, start_symbol):
    """
    For simplicity, a Greedy Decoder is Beam search when K=1. This is necessary for inference as we don't know the
    target sequence input. Therefore we try to generate the target input word by word, then feed it into the transformer.
    Starting Reference: http://nlp.seas.harvard.edu/2018/04/03/attention.html#greedy-decoding
    :param model: Transformer Model
    :param enc_input: The encoder input
    :param start_symbol: The start symbol. In this example it is 'S' which corresponds to index 4
    :return: The target input
    """
    enc_outputs, enc_self_attns = model.encoder(enc_input)
    dec_input = torch.zeros(1, 0).type_as(enc_input.data)
    terminal = False
    next_symbol = start_symbol
    while not terminal:         
        dec_input = torch.cat([dec_input.detach(),torch.tensor([[next_symbol]],dtype=enc_input.dtype).cuda()],-1)
        dec_outputs, _, _ = model.decoder(dec_input, enc_input, enc_outputs)
        projected = model.projection(dec_outputs)
        prob = projected.squeeze(0).max(dim=-1, keepdim=False)[1]
        next_word = prob.data[-1]
        next_symbol = next_word
        if next_symbol == tgt_vocab["."]:
            terminal = True
        print(next_word)            
    return dec_input



# test
enc_inputs, _, _ = next(iter(loader))
enc_inputs = enc_inputs.cuda()
for i in range(len(enc_inputs)):
    greedy_dec_input = greedy_decoder(model, enc_inputs[i].view(1, -1), start_symbol=tgt_vocab["S"])
    predict, _, _, _ = model(enc_inputs[i].view(1, -1), greedy_dec_input)
    predict = predict.data.max(1, keepdim=True)[1]
    print(enc_inputs[i], '->', [idx2word[n.item()] for n in predict.squeeze()])

tensor(1, device='cuda:0')
tensor(2, device='cuda:0')
tensor(3, device='cuda:0')
tensor(5, device='cuda:0')
tensor(8, device='cuda:0')
tensor([1, 2, 3, 5, 0], device='cuda:0') -> ['i', 'want', 'a', 'coke', '.']
tensor(1, device='cuda:0')
tensor(2, device='cuda:0')
tensor(3, device='cuda:0')
tensor(4, device='cuda:0')
tensor(8, device='cuda:0')
tensor([1, 2, 3, 4, 0], device='cuda:0') -> ['i', 'want', 'a', 'beer', '.']
